In [31]:
import torch
CUDA = torch.cuda.is_available()
print(CUDA)

False


In [76]:
import time
from lifelines.utils import concordance_index 
import sys
from torch import nn
import survival_analysis_chirag
import numpy as np
import pandas as pd
import network
from torch.utils.data import TensorDataset, Dataset
import torch.utils.data.dataloader as dataloader
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


ds = pd.read_csv('./datasets/whas500.csv',sep=',')
# ds = pd.read_csv('./datasets/whas1638.csv',sep=',')
train = ds
# train = ds[:400]
# validation = ds[400:]

x = train[['age', 'gender', 'bmi', 'chf', 'miord']]
# x = train[['1', '2', '3', '4', '5']]
e = train['fstat']
t = train['lenfol']

x = torch.from_numpy(x.as_matrix()).float()
e = torch.from_numpy(e.as_matrix()).float()
t = torch.from_numpy(t.as_matrix())

In [77]:
if CUDA:
    x = x.cuda()
    e = e.cuda()
    t = t.cuda()

In [78]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
#         m.weight.data.fill_(0)
#         m.bias.data.fill_(1)

def init_weights_for_cox(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0)
        m.bias.data.fill_(0)

In [79]:
def train(model, x, e, t, CUDA, optimizer, n_epochs):

    # Initialize Metrics
    c_index = []
    valid_c_index = []
    diff = 1e-3
    
    risk_set = []
    prev_loss = 0
    for i in range(len(t)):
        temp = []
        for j in range(len(t)):
            if t[j] >= t[i]:
                temp.append(j)
        risk_set.append(temp)
        
    start = time.time()
    for epoch in range(n_epochs):

        optimizer.zero_grad()
        # print("x: ", x)
        outputs = model(x)

        loss = negative_log_likelihood(outputs, e, risk_set, CUDA)
        loss.backward()
        optimizer.step()


        my_loss = loss.cpu().data.numpy()
        if abs(my_loss - prev_loss) < diff:
            break
        prev_loss = my_loss
        
        ci_train = get_concordance_index(outputs, t, e)
        c_index.append(ci_train)
        torch.cuda.empty_cache()
                   
        print('Finished Training with %d iterations in %0.2fs' % (epoch + 1, time.time() - start))
    
    metrics = {}
    metrics['c-index'] = c_index
    return metrics


In [80]:
def negative_log_likelihood(risk, E, risk_set, CUDA):
    
    import numpy as np
    
#     new_risk = []
#     for i in range(len(risk_set)):
#         new_risk.append(risk[risk_set[i]])
        
#     log_risk = []
#     for i in range(len(new_risk)):
#         temp = torch.logsumexp(new_risk[i], 0)
#         log_risk.append(temp)

    lnumerator = risk
    
    idxs = range(risk.shape[0])
    
    
    ldenominator = []
    
    for i in range(len(idxs)):
        
        ldenominator.append(torch.logsumexp(risk[risk_set[i]], dim=0))
    
    ldenominator = torch.cat(ldenominator).reshape(-1, 1)
    
    
    likelihoods = lnumerator - ldenominator
    
    E =  np.where(E.cpu().data.numpy()==1)[0]
    

    
    neg_likelihood = - torch.sum(likelihoods[E])
    


    return neg_likelihood


In [81]:
def get_concordance_index(x, t, e, **kwargs):
    x = x.detach().cpu().numpy()
    t = t.detach().cpu().numpy()
    e = e.detach().cpu().numpy()
    computed_hazard = np.exp(x)

    return concordance_index(t,-1*computed_hazard,e)


In [82]:

# For CPH, set cox argument as True
print("CPH model")
n_in = x.shape[1]

layers_sizes = [n_in, 1]

# Construct Neural Network
layers = []
for i in range(len(layers_sizes)-2):
    layers.append(nn.Linear(layers_sizes[i],layers_sizes[i+1]))
    layers.append(nn.ReLU())

layers.append(nn.Linear(layers_sizes[-2], layers_sizes[-1]))
my_network = nn.Sequential(*layers)
#my_network.apply(init_weights)

#optimizer = optimizer = torch.optim.SGD(my_network.parameters(), lr=learning_rate, momentum=momentum, weight_decay=L2_reg, nesterov=True)

optimizer = torch.optim.Adam(my_network.parameters(), lr=1e-3)
my_network.train()
if CUDA:
    my_network.cuda()

# If you have validation data, you can add it as the valid_dataloader parameter to the function
n_epochs = 5000
metrics = train(my_network, x, e, t, CUDA, optimizer, n_epochs)
print()

print("Done")

CPH model
Finished Training with 1 iterations in 0.05s
Finished Training with 2 iterations in 0.08s
Finished Training with 3 iterations in 0.12s
Finished Training with 4 iterations in 0.16s
Finished Training with 5 iterations in 0.19s
Finished Training with 6 iterations in 0.23s
Finished Training with 7 iterations in 0.27s
Finished Training with 8 iterations in 0.31s
Finished Training with 9 iterations in 0.35s
Finished Training with 10 iterations in 0.38s
Finished Training with 11 iterations in 0.42s
Finished Training with 12 iterations in 0.45s
Finished Training with 13 iterations in 0.49s
Finished Training with 14 iterations in 0.53s
Finished Training with 15 iterations in 0.57s
Finished Training with 16 iterations in 0.60s
Finished Training with 17 iterations in 0.64s
Finished Training with 18 iterations in 0.67s
Finished Training with 19 iterations in 0.72s
Finished Training with 20 iterations in 0.76s
Finished Training with 21 iterations in 0.80s
Finished Training with 22 iterati

Finished Training with 180 iterations in 7.28s
Finished Training with 181 iterations in 7.32s
Finished Training with 182 iterations in 7.36s
Finished Training with 183 iterations in 7.40s
Finished Training with 184 iterations in 7.43s
Finished Training with 185 iterations in 7.47s
Finished Training with 186 iterations in 7.52s
Finished Training with 187 iterations in 7.57s
Finished Training with 188 iterations in 7.60s
Finished Training with 189 iterations in 7.64s
Finished Training with 190 iterations in 7.68s
Finished Training with 191 iterations in 7.72s
Finished Training with 192 iterations in 7.77s
Finished Training with 193 iterations in 7.80s
Finished Training with 194 iterations in 7.84s
Finished Training with 195 iterations in 7.88s
Finished Training with 196 iterations in 7.92s
Finished Training with 197 iterations in 7.96s
Finished Training with 198 iterations in 8.00s
Finished Training with 199 iterations in 8.04s
Finished Training with 200 iterations in 8.08s
Finished Trai

Finished Training with 357 iterations in 14.40s
Finished Training with 358 iterations in 14.44s
Finished Training with 359 iterations in 14.48s
Finished Training with 360 iterations in 14.52s
Finished Training with 361 iterations in 14.56s
Finished Training with 362 iterations in 14.60s
Finished Training with 363 iterations in 14.64s
Finished Training with 364 iterations in 14.68s
Finished Training with 365 iterations in 14.72s
Finished Training with 366 iterations in 14.76s
Finished Training with 367 iterations in 14.80s
Finished Training with 368 iterations in 14.84s
Finished Training with 369 iterations in 14.88s
Finished Training with 370 iterations in 14.92s
Finished Training with 371 iterations in 14.97s
Finished Training with 372 iterations in 15.01s
Finished Training with 373 iterations in 15.05s
Finished Training with 374 iterations in 15.10s
Finished Training with 375 iterations in 15.15s
Finished Training with 376 iterations in 15.19s
Finished Training with 377 iterations in

Finished Training with 533 iterations in 21.60s
Finished Training with 534 iterations in 21.64s
Finished Training with 535 iterations in 21.68s
Finished Training with 536 iterations in 21.72s
Finished Training with 537 iterations in 21.75s
Finished Training with 538 iterations in 21.79s
Finished Training with 539 iterations in 21.84s
Finished Training with 540 iterations in 21.88s
Finished Training with 541 iterations in 21.92s
Finished Training with 542 iterations in 21.96s
Finished Training with 543 iterations in 21.99s
Finished Training with 544 iterations in 22.03s
Finished Training with 545 iterations in 22.08s
Finished Training with 546 iterations in 22.11s
Finished Training with 547 iterations in 22.15s
Finished Training with 548 iterations in 22.19s
Finished Training with 549 iterations in 22.23s
Finished Training with 550 iterations in 22.27s
Finished Training with 551 iterations in 22.32s
Finished Training with 552 iterations in 22.36s
Finished Training with 553 iterations in

Finished Training with 707 iterations in 28.75s
Finished Training with 708 iterations in 28.79s
Finished Training with 709 iterations in 28.83s
Finished Training with 710 iterations in 28.87s
Finished Training with 711 iterations in 28.91s
Finished Training with 712 iterations in 28.96s
Finished Training with 713 iterations in 29.01s
Finished Training with 714 iterations in 29.05s
Finished Training with 715 iterations in 29.09s
Finished Training with 716 iterations in 29.13s
Finished Training with 717 iterations in 29.16s
Finished Training with 718 iterations in 29.21s
Finished Training with 719 iterations in 29.25s
Finished Training with 720 iterations in 29.30s
Finished Training with 721 iterations in 29.34s
Finished Training with 722 iterations in 29.38s
Finished Training with 723 iterations in 29.42s
Finished Training with 724 iterations in 29.47s
Finished Training with 725 iterations in 29.51s
Finished Training with 726 iterations in 29.55s
Finished Training with 727 iterations in

...

Finished Training with 1052 iterations in 44.60s
Finished Training with 1053 iterations in 44.64s
Finished Training with 1054 iterations in 44.68s
Finished Training with 1055 iterations in 44.72s
Finished Training with 1056 iterations in 44.76s
Finished Training with 1057 iterations in 44.80s
Finished Training with 1058 iterations in 44.85s
Finished Training with 1059 iterations in 44.89s
Finished Training with 1060 iterations in 44.93s
Finished Training with 1061 iterations in 44.97s
Finished Training with 1062 iterations in 45.02s
Finished Training with 1063 iterations in 45.07s
Finished Training with 1064 iterations in 45.11s
Finished Training with 1065 iterations in 45.15s
Finished Training with 1066 iterations in 45.20s
Finished Training with 1067 iterations in 45.25s
Finished Training with 1068 iterations in 45.29s
Finished Training with 1069 iterations in 45.33s
Finished Training with 1070 iterations in 45.37s
Finished Training with 1071 iterations in 45.41s
Finished Training wi

Finished Training with 1223 iterations in 52.16s
Finished Training with 1224 iterations in 52.20s
Finished Training with 1225 iterations in 52.24s
Finished Training with 1226 iterations in 52.29s
Finished Training with 1227 iterations in 52.34s
Finished Training with 1228 iterations in 52.39s
Finished Training with 1229 iterations in 52.44s
Finished Training with 1230 iterations in 52.48s
Finished Training with 1231 iterations in 52.53s
Finished Training with 1232 iterations in 52.57s
Finished Training with 1233 iterations in 52.61s
Finished Training with 1234 iterations in 52.65s
Finished Training with 1235 iterations in 52.69s
Finished Training with 1236 iterations in 52.73s
Finished Training with 1237 iterations in 52.77s
Finished Training with 1238 iterations in 52.81s
Finished Training with 1239 iterations in 52.86s
Finished Training with 1240 iterations in 52.90s
Finished Training with 1241 iterations in 52.94s
Finished Training with 1242 iterations in 52.98s
Finished Training wi

Finished Training with 1393 iterations in 59.28s
Finished Training with 1394 iterations in 59.32s
Finished Training with 1395 iterations in 59.36s
Finished Training with 1396 iterations in 59.41s
Finished Training with 1397 iterations in 59.45s
Finished Training with 1398 iterations in 59.50s
Finished Training with 1399 iterations in 59.54s
Finished Training with 1400 iterations in 59.58s
Finished Training with 1401 iterations in 59.61s
Finished Training with 1402 iterations in 59.65s
Finished Training with 1403 iterations in 59.69s
Finished Training with 1404 iterations in 59.73s
Finished Training with 1405 iterations in 59.77s
Finished Training with 1406 iterations in 59.81s
Finished Training with 1407 iterations in 59.85s
Finished Training with 1408 iterations in 59.89s
Finished Training with 1409 iterations in 59.92s
Finished Training with 1410 iterations in 59.97s
Finished Training with 1411 iterations in 60.01s
Finished Training with 1412 iterations in 60.05s
Finished Training wi

Finished Training with 1566 iterations in 66.40s
Finished Training with 1567 iterations in 66.44s
Finished Training with 1568 iterations in 66.48s
Finished Training with 1569 iterations in 66.52s
Finished Training with 1570 iterations in 66.56s
Finished Training with 1571 iterations in 66.60s
Finished Training with 1572 iterations in 66.65s
Finished Training with 1573 iterations in 66.69s
Finished Training with 1574 iterations in 66.73s
Finished Training with 1575 iterations in 66.79s
Finished Training with 1576 iterations in 66.83s
Finished Training with 1577 iterations in 66.87s
Finished Training with 1578 iterations in 66.91s
Finished Training with 1579 iterations in 66.96s
Finished Training with 1580 iterations in 67.00s
Finished Training with 1581 iterations in 67.04s
Finished Training with 1582 iterations in 67.08s
Finished Training with 1583 iterations in 67.12s
Finished Training with 1584 iterations in 67.17s
Finished Training with 1585 iterations in 67.21s
Finished Training wi

Finished Training with 1738 iterations in 73.45s
Finished Training with 1739 iterations in 73.49s
Finished Training with 1740 iterations in 73.53s
Finished Training with 1741 iterations in 73.57s
Finished Training with 1742 iterations in 73.61s
Finished Training with 1743 iterations in 73.65s
Finished Training with 1744 iterations in 73.69s
Finished Training with 1745 iterations in 73.73s
Finished Training with 1746 iterations in 73.77s
Finished Training with 1747 iterations in 73.80s
Finished Training with 1748 iterations in 73.84s
Finished Training with 1749 iterations in 73.88s
Finished Training with 1750 iterations in 73.93s
Finished Training with 1751 iterations in 73.97s
Finished Training with 1752 iterations in 74.00s
Finished Training with 1753 iterations in 74.04s
Finished Training with 1754 iterations in 74.08s
Finished Training with 1755 iterations in 74.12s
Finished Training with 1756 iterations in 74.17s
Finished Training with 1757 iterations in 74.20s
Finished Training wi

In [83]:
my_network

Sequential(
  (0): Linear(in_features=5, out_features=1, bias=True)
)

In [84]:
print("Train C-Index:", metrics['c-index'])

Train C-Index: [0.6754314761340803, 0.6753516347522921, 0.6752984071644333, 0.6751520312978216, 0.6749125071524571, 0.6746862899040573, 0.674526607140481, 0.6744467657586928, 0.674260469201187, 0.67412740023154, 0.6741007864376106, 0.6742205485102929, 0.6743403105829752, 0.6742870829951164, 0.6744467657586928, 0.6747794381828102, 0.67500565543121, 0.6753915554431862, 0.675750841661233, 0.6762431968489268, 0.6765625623760795, 0.6770416106668086, 0.6776271141332553, 0.6782392313936313, 0.6788912693449015, 0.6793570107386658, 0.6799957417929713, 0.6805679383624532, 0.6813397383864057, 0.6822579142769698, 0.6830031005069928, 0.6833490798280749, 0.6839079695005922, 0.6844934729670388, 0.6849326005668738, 0.6851721247122383, 0.6857975488695791, 0.6862100626754847, 0.6867157247601432, 0.6871149316690841, 0.6875673661658838, 0.6879931868687541, 0.6883790868807302, 0.6889512834502123, 0.6893238765652238, 0.6896964696802352, 0.6902154386618584, 0.6906279524677641, 0.6909073973040227, 0.691173535